In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.conf import SparkConf

In [ ]:
spark = SparkSession.builder.appName("session-data").getOrCreate()

In [ ]:
df = spark.read.format("csv").option("header","true").option("inferSchema",True).load("./students.csv")
df.show(20)

In [ ]:
df = df.withColumn('amount',F.col('amount').cast('int'))

In [ ]:
df.printSchema()

In [ ]:
df = df.groupBy("first").agg(F.sum("amount").alias("total_amount"))

In [ ]:
df_avg = df.select(F.avg('total_amount').alias('avg_amount'))

In [ ]:
df_avg.show()

In [ ]:
df = df.crossJoin(df_avg)

In [ ]:
df = df.withColumn('compared_to_Avg',F.round(F.col('total_amount')/F.col('avg_amount'),3))

In [ ]:
df.show()

In [ ]:
df.write.mode('overwrite').partitionBy('compared_to_Avg').parquet('./output_data_parquet/')

In [ ]:
# After Optimizing the code in application level

In [ ]:
spark_conf = SparkConf()

In [ ]:
spark_conf.set('spark.sql.adaptive.coalescePartitions.initialPartitionNum', 10)

In [ ]:
spark_conf.set('spark.sql.adaptive.coalescePartitions.parallelismFirst', 'false')

In [ ]:
spark_conf.set('spark.sql.files.minPartitionNum',1)

In [ ]:
spark_conf.set('spark.sql.files.maxPartitionBytes','3mb')

In [ ]:
spark = SparkSession.builder.appName("optimisation").getOrCreate()

In [ ]:
df = spark.read.format("csv").option("header","true").option("inferSchema",True).load("students.csv")

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn("amount",F.col("amount").cast("int"))

In [ ]:
df = df.groupBy("first").agg(F.sum("amount").alias("total_amount"))

In [ ]:
df_avg = df.select(F.avg("total_amount").alias("avg_amount"))

In [ ]:
df = df.crossJoin(df_avg)

In [ ]:
df = df.withColumn("compared_to_avg",F.round(F.col("total_amount")/F.col("avg_amount"),3))

In [ ]:
df.write.mode("overwrite").partitionBy("compared_to_avg").parquet("output_folder")